In [1]:
import tensorflow as tf

# 定义神经网络结构的相关参数
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

In [2]:
def get_weight_variable(shape, regularizer):
    '''通过tf.get_variable函数来获取变量'''
    #print("name:",tf.get_variable_scope().name)
    weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    # 当给出了正则化生成函数时，将当前变量的正则化损失加入名字为losses的集合
    if regularizer != None:
        tf.add_to_collection('losses', regularizer(weights))
    return weights

In [3]:
def inference(input_tensor, regularizer):
    '''定义神经网络的前向传播过程'''
    # 声明第一层神经网络的变量并完成前向传播过程
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([INPUT_NODE,LAYER1_NODE],regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
    
    # 声明第二层
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE,OUTPUT_NODE],regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    
    for ele1 in tf.trainable_variables():  
        print(ele1.name)
    return layer2

In [21]:
import os

from tensorflow.examples.tutorials.mnist import input_data


# 配置神经网络中的参数
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

# 模型保存的路径和文件名
MODEL_SAVE_PATH = "model/"
MODEL_NAME = "model.ckpt"


In [22]:
def train(mnist):
    
    # 清理图中保存的变量，重置图
    tf.reset_default_graph()
    # 定义输入输出
    
    x = tf.placeholder(tf.float32, [None,INPUT_NODE],name="x-input")
    y_ = tf.placeholder(tf.float32, [None,OUTPUT_NODE],name="y-input")

    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    
    # 直接使用inference中定义的前向传播过程
    y = inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)
    
    # 定义损失函数、学习率、滑动平均操作以及训练过程
    
    #print("train name:",tf.get_variable_scope().name)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY,staircase=True)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')   
    
    #初始化Tensorflow持久化类
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x:xs,y_:ys})
            
            # 每1000轮保存一次模型
            if i % 1000 == 0:
                print(loss_value, step)
                # 输出在当前batch上的损失
                print("After " + str(step) +" training steps, loss on training batch is " +str(loss_value)+ ". ")
                saver.save(sess, os.path.join(MODEL_SAVE_PATH,MODEL_NAME), global_step = global_step)



In [23]:
def main():
    mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
    train(mnist)

In [24]:
main()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
layer1/weights:0
layer1/biases:0
layer2/weights:0
layer2/biases:0
2.885 1
After 1 training steps, loss on training batch is 2.885. 
0.309594 1001
After 1001 training steps, loss on training batch is 0.309594. 
0.220914 2001
After 2001 training steps, loss on training batch is 0.220914. 
0.151756 3001
After 3001 training steps, loss on training batch is 0.151756. 
0.131714 4001
After 4001 training steps, loss on training batch is 0.131714. 
0.116761 5001
After 5001 training steps, loss on training batch is 0.116761. 
0.093061 6001
After 6001 training steps, loss on training batch is 0.093061. 
0.0888082 7001
After 7001 training steps, loss on training batch is 0.0888082. 
0.0815856 8001
After 8001 training steps, loss on training batch is 0.0815856. 
0.0834794 9001
After 9001 training steps, loss

**注意**：多次运行调试程序的话需要在train()函数前加`tf.reset_default_graph()`以重置图，清理已存在的变量，否则会报错。